In [2]:
!pip install pyenchant

  Using cached https://files.pythonhosted.org/packages/9e/54/04d88a59efa33fefb88133ceb638cdf754319030c28aadc5a379d82140ed/pyenchant-2.0.0.tar.gz
    Complete output from command python setup.py egg_info:
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "C:\Users\SAHIBN~1\AppData\Local\Temp\pip-install-_1qmrg3x\pyenchant\setup.py", line 212, in <module>
        import enchant
      File "C:\Users\SAHIBN~1\AppData\Local\Temp\pip-install-_1qmrg3x\pyenchant\enchant\__init__.py", line 92, in <module>
        from enchant import _enchant as _e
      File "C:\Users\SAHIBN~1\AppData\Local\Temp\pip-install-_1qmrg3x\pyenchant\enchant\_enchant.py", line 145, in <module>
        raise ImportError(msg)
    ImportError: The 'enchant' C library was not found. Please install it via your OS package manager, or use a pre-built binary wheel from PyPI.
    
    ----------------------------------------


Command "python setup.py egg_info" failed with error code 1 in C:\Users\SAHIBN~1\AppData\Local\Temp\pip-install-_1qmrg3x\pyenchant\


# Import Packages

In [9]:
import pandas as pd
import numpy as np
import re
import enchant
import fileinput

from collections import Counter
from sklearn.model_selection import train_test_split

#importing keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM, Input, SpatialDropout1D
from keras import utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.optimizers import SGD

# nltk
import nltk
from nltk.corpus import stopwords
from nltk.corpus import words
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

ModuleNotFoundError: No module named 'enchant'

# Import Data

In [3]:
# TEXT CLENAING
TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

#Load Dataset
df = pd.read_csv("data.csv")

#converting into small letters
for line in fileinput.input("data.csv",inplace=1):
    print(line.lower(),end='')

# Preproccessing

In [4]:
#removing stopwords
nltk.download('stopwords')
stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /home/nbuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
def preprocess(text, stem=False):
    # Remove link,user and special characters
    text = re.sub(TEXT_CLEANING_RE, ' ', str(tweet_content).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

d = enchant.Dict("en_US")
for i in range(len(df)):
    string = df.tweet_content[i]
    english_words = []
    for word in string.split():
        if d.check(word):
            english_words.append(word)
            #print=" ".join(english_words)
            vin=" ".join(english_words)
    df.tweet_content[i]=vin

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
def tokenize(untokenized):
    tokenized = sent_tokenize(untokenized)
    
    return tokenized

def shuffle_tokenize(test):
    random.shuffle(text)
    newl=list(text)
    shuffled.append(newl)
    
    return text


augmented = []
reps = []

for neg in df.tweet_content:
    if df.harassment == '1' or df.harassment == 1:
        txt = tokenize(neg)

### The following process is an alternative to One-Hot Encoding

In [8]:
all_text2 = ' '.join(df.tweet_content)
# create a list of words
words = all_text2.split()

# Count all the words using Counter Method
count_words = Counter(words)

total_words = len(words)
sorted_words = count_words.most_common(total_words)

# Assigning a unique index value to all the words
vocab_to_int = {w:i+1 for i, (w,c) in enumerate(sorted_words)}

tweets_int = []
for tweet in df.tweet_content:
    r = [vocab_to_int[w] for w in tweet.split()]
    tweets_int.append(r)
print (tweets_int[0:2])

tweets_len = [len(x) for x in tweets_int]
tweets_int = [ tweets_int[i] for i, l in enumerate(tweets_len) if l>0 ]

x1=pad_sequences(tweets_int, maxlen=22, padding='post')

[[138, 24, 90, 485, 122, 558, 309, 7, 270, 44, 648, 68], [8, 193, 4168, 427, 14, 2, 4169, 63, 56, 169, 6, 1782, 11]]


# Build Model

In [88]:
y = df.iloc[:,2].values
X_train, X_test, y_train, y_test = train_test_split(x1, y, test_size=0.1, random_state=42)

batch_size = 32
num_classes = 2
epochs = 10


model = Sequential()
model.add(Embedding(17207, 256))
model.add(Dense(128))
model.add(Dropout(0.5))
model.add(LSTM(64, dropout=0.9, recurrent_dropout=0.5))
model.add(Dense(1, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_30 (Embedding)     (None, None, 256)         4404992   
_________________________________________________________________
dense_30 (Dense)             (None, None, 128)         32896     
_________________________________________________________________
dropout_14 (Dropout)         (None, None, 128)         0         
_________________________________________________________________
lstm_31 (LSTM)               (None, 64)                49408     
_________________________________________________________________
dense_31 (Dense)             (None, 1)                 65        
Total params: 4,487,361
Trainable params: 4,487,361
Non-trainable params: 0
_________________________________________________________________


# Compile Model

In [89]:
model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

# Train Model

In [90]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))

Train on 5736 samples, validate on 638 samples
Epoch 1/10
5736/5736 [==============================] - 29s 5ms/step - loss: 0.6800 - acc: 0.5757 - val_loss: 0.5438 - val_acc: 0.7555
Epoch 2/10
5736/5736 [==============================] - 27s 5ms/step - loss: 0.4964 - acc: 0.7831 - val_loss: 0.3690 - val_acc: 0.8636
Epoch 3/10
5736/5736 [==============================] - 20s 3ms/step - loss: 0.3714 - acc: 0.8567 - val_loss: 0.3329 - val_acc: 0.8824
Epoch 4/10
5736/5736 [==============================] - 20s 3ms/step - loss: 0.3098 - acc: 0.8882 - val_loss: 0.3675 - val_acc: 0.8777
Epoch 5/10
5736/5736 [==============================] - 21s 4ms/step - loss: 0.2651 - acc: 0.9059 - val_loss: 0.3843 - val_acc: 0.8668
Epoch 6/10
5736/5736 [==============================] - 21s 4ms/step - loss: 0.2570 - acc: 0.9067 - val_loss: 0.4058 - val_acc: 0.8621
Epoch 7/10
5736/5736 [==============================] - 21s 4ms/step - loss: 0.2311 - acc: 0.9195 - val_loss: 0.3899 - val_acc: 0.8605
Epoch 8/

## Output

In [87]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.45439859150345424
Test accuracy: 0.8150470226909673
